In [1]:
import sys
import os
from pathlib import Path
project_root = Path.cwd().parent
sys.path.append(str(project_root))

from src.config.config_loader import ConfigLoader
from src.config.factory import ServiceFactory

c:\Users\giaco\.conda\envs\nl2sql\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\giaco\.conda\envs\nl2sql\lib\site-packages\snowflake\sqlalchemy\base.py:1068: SAWarning: The GenericFunction 'flatten' is already registered and is going to be overridden.
  functions.register_function("flatten", flatten)


In [2]:
config_path = os.path.join(project_root, 'config.yaml')
config = ConfigLoader.load_config(config_path)
print(config)

AppConfig(database=DatabaseConfig(type='postgres', host='localhost', port=5432, database='postgres', user='postgres', password='admin', schema='video_games', warehouse=None, account=None, role=None), llm=LLMConfig(type='ollama', api_key=None, model='llama3.1', base_url='http://localhost:11434', language=<SupportedLanguage.ENGLISH: 'english'>), prompt=PromptConfig(include_sample_data=True, max_sample_rows=3), vector_store=None, debug=True)


In [3]:
db = ServiceFactory.create_db_connector(config.database)
if not db.connect():
    raise RuntimeError("Failed to connect to database")

In [4]:
metadata_retriever = ServiceFactory.create_metadata_retriever(config.database, db)

In [9]:
er_diagram = metadata_retriever.get_er_diagram()

print("Tabelle nel diagramma:")
for table in er_diagram.tables:
    print(f"Tabella: {table.name}")
    print("Colonne:")
    for col in table.columns:
        flags = []
        if col.is_primary_key: flags.append("PK")
        if col.is_foreign_key: flags.append("FK")
        if col.is_nullable: flags.append("NULL")
        flags_str = f" [{', '.join(flags)}]" if flags else ""
        print(f" - {col.name} ({col.type}{flags_str})") 
        
for rel in er_diagram.relationships:
    print(f"  {rel.from_table}.{rel.from_column} -> {rel.to_table}.{rel.to_column} ({rel.relationship_type})")

Tabelle nel diagramma:
Tabella: genre
Colonne:
 - id (INTEGER [PK])
 - genre_name (VARCHAR(50) [NULL])
Tabella: publisher
Colonne:
 - id (INTEGER [PK])
 - publisher_name (VARCHAR(100) [NULL])
Tabella: platform
Colonne:
 - id (INTEGER [PK])
 - platform_name (VARCHAR(50) [NULL])
Tabella: region
Colonne:
 - id (INTEGER [PK])
 - region_name (VARCHAR(50) [NULL])
Tabella: game
Colonne:
 - id (INTEGER [PK])
 - genre_id (INTEGER [FK, NULL])
 - game_name (VARCHAR(200) [NULL])
Tabella: game_publisher
Colonne:
 - id (INTEGER [PK])
 - game_id (INTEGER [FK, NULL])
 - publisher_id (INTEGER [FK, NULL])
Tabella: game_platform
Colonne:
 - id (INTEGER [PK])
 - game_publisher_id (INTEGER [FK, NULL])
 - platform_id (INTEGER [FK, NULL])
 - release_year (INTEGER [NULL])
Tabella: region_sales
Colonne:
 - region_id (INTEGER [FK, NULL])
 - game_platform_id (INTEGER [FK, NULL])
 - num_sales (NUMERIC(5, 2) [NULL])
  game.genre_id -> genre.id (N-1)
  game_publisher.game_id -> game.id (N-1)
  game_publisher.publis

In [10]:
er_metadata = metadata_retriever.get_er_metadata()

import json
print(json.dumps(er_metadata, indent=2))

{
  "tables": [
    {
      "name": "genre",
      "columns": [
        {
          "name": "id",
          "type": "INTEGER",
          "isPrimaryKey": true,
          "isForeignKey": false,
          "isNullable": false
        },
        {
          "name": "genre_name",
          "type": "VARCHAR(50)",
          "isPrimaryKey": false,
          "isForeignKey": false,
          "isNullable": true
        }
      ],
      "position": {
        "x": 0,
        "y": 0
      }
    },
    {
      "name": "publisher",
      "columns": [
        {
          "name": "id",
          "type": "INTEGER",
          "isPrimaryKey": true,
          "isForeignKey": false,
          "isNullable": false
        },
        {
          "name": "publisher_name",
          "type": "VARCHAR(100)",
          "isPrimaryKey": false,
          "isForeignKey": false,
          "isNullable": true
        }
      ],
      "position": {
        "x": 0,
        "y": 0
      }
    },
    {
      "name": "platform",

In [11]:
api_response = {
    "success": True,
    "data": er_metadata
}
print(json.dumps(api_response, indent=2))

{
  "success": true,
  "data": {
    "tables": [
      {
        "name": "genre",
        "columns": [
          {
            "name": "id",
            "type": "INTEGER",
            "isPrimaryKey": true,
            "isForeignKey": false,
            "isNullable": false
          },
          {
            "name": "genre_name",
            "type": "VARCHAR(50)",
            "isPrimaryKey": false,
            "isForeignKey": false,
            "isNullable": true
          }
        ],
        "position": {
          "x": 0,
          "y": 0
        }
      },
      {
        "name": "publisher",
        "columns": [
          {
            "name": "id",
            "type": "INTEGER",
            "isPrimaryKey": true,
            "isForeignKey": false,
            "isNullable": false
          },
          {
            "name": "publisher_name",
            "type": "VARCHAR(100)",
            "isPrimaryKey": false,
            "isForeignKey": false,
            "isNullable": true
     